In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from math import sin, cos, sqrt, atan2, radians
from geopy.distance import vincenty
from sklearn import preprocessing
from scipy import interpolate
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
import networkx as nx
# First, import the data:
pd.options.display.max_columns = 100
# pd.options.display.max_rows = None


## Adding a month of data - Jan 2016 - Feb 2016


list_of_df_names = ['fahrzeiten_soll_ist_20160103_20160109', 'fahrzeiten_soll_ist_20160110_20160116',
                    'fahrzeiten_soll_ist_20160117_20160123', 'fahrzeiten_soll_ist_20160124_20160130',
                    'fahrzeiten_soll_ist_20160131_20160206', 'fahrzeiten_soll_ist_20160207_20160213',
                    'fahrzeiten_soll_ist_20160214_20160220', 'fahrzeiten_soll_ist_20160221_20160227']

# Just the first two weeks

list_of_df_names = ['fahrzeiten_soll_ist_20160103_20160109', 'fahrzeiten_soll_ist_20160110_20160116']

list_of_df = []
for i in list_of_df_names:
    list_of_df.append(pd.read_csv(i+'.csv'))
df_1 = pd.concat(list_of_df, ignore_index=True)
list_of_df.clear()


####################
df_haltepunkt = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltepunkt.csv")
# Fix the gps
# print(type(df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))))
# df_haltepunkt['GPS_Latitude'] = df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))
# df_haltepunkt['GPS_Longitude'] = df_haltepunkt['GPS_Longitude'].apply(lambda x: float(x.replace(',', '.')))


# Bus Station
df_haltestelle = pd.read_csv("../Travel times 2016 of the VBZ in the target-performance comparison/haltestelle.csv")
####################

def find_dist(lat1, lon1, lat_2, lon2):
    c1 = (lat1, lon1)
    c2 = (lat_2, lon2)
    return geopy.distance.vincenty(c1, c2).km
####################
# Some useful atts #
####################

# Target attributes - from: in seconds
target_arrival_from = "soll_an_von"

# Actual attributes - from: in seconds
actual_arrival_from = "ist_an_von"

# nominal departure from in seconds
target_departure_from = "soll_ab_von"

# actual  departure from in seconds
actual_departure_from = "ist_ab_von"
#######################################

#######################################
# Target attributes - from: in seconds
target_arrival_to = "soll_an_nach"

# Actual attributes - from: in seconds
actual_arrival_to = "ist_an_nach1"

# nominal departure from in seconds
target_departure_to = "soll_ab_nach"

# actual  departure from in seconds
actual_departure_to = "ist_ab_nach"
#####################################

#####################################


In [2]:
df_haltepunkt.dropna(axis=0, how='any', inplace=True, subset=['GPS_Longitude'])


In [3]:
# Fix the gps
# print(type(df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))))
df_haltepunkt['GPS_Latitude'] = df_haltepunkt['GPS_Latitude'].apply(lambda x: float(x.replace(',', '.')))
df_haltepunkt['GPS_Longitude'] = df_haltepunkt['GPS_Longitude'].apply(lambda x: float(x.replace(',', '.')))


In [4]:
len(df_1)

2804468

In [5]:
# Merge the halt punkt from


df_1['halt_punkt_id'] = df_1['halt_punkt_id_von']
df_1 = df_1.merge(df_haltepunkt, on=['halt_punkt_id'])


In [6]:
# df_1['GPS_Latitude_from'] = df_1['GPS_Latitude']
# df_1['GPS_Longitude_to'] = df_1['GPS_Longitude']

# Merge punkt to

df_1['halt_punkt_id'] = df_1['halt_punkt_id_nach']
df_1 = df_1.merge(df_haltepunkt, on='halt_punkt_id')


In [7]:

df_1['halt_id'] = df_1['halt_id_x']
df_1 = df_1.merge(df_haltestelle, on='halt_id')

In [8]:
df_1['halt_id'] = df_1['halt_id_y']
df_1 = df_1.merge(df_haltestelle, on='halt_id')

In [9]:
# df_1.info()

In [10]:
# df_1.sort_values(by = ['betriebsdatum', 'soll_ab_von', 'fahrt_id', 'seq_von'], inplace=True)
df_1 = df_1[df_1['betriebsdatum'] == df_1['datum_nach']]

In [11]:
df_1 = df_1[(df_1['soll_an_nach'] > 18000) & (df_1['soll_an_nach'] < 86400)]
# df_1

In [12]:
# Some Signals
target_stationary = 'target_stationary'
target_travel_time = 'target_travel_time'
delay = 'delay'

actual_stationary = 'actual_stationary'
actual_travel_time = 'actual_travel_time'

df_1[target_stationary] = df_1[target_departure_from] - df_1[target_arrival_from]
df_1[target_travel_time] = df_1[target_arrival_to] - df_1[target_departure_from]
df_1[delay] = df_1[actual_arrival_from] - df_1[target_arrival_from]


# Signal - 1 - actual arrival - actual departure
df_1[actual_stationary] = df_1[actual_departure_from] - df_1[actual_arrival_from]
df_1[actual_travel_time] = df_1[actual_arrival_to] - df_1[actual_departure_from]

# b[target_stationary] = b[target_departure_from] - b[target_arrival_from]
# b['actual'] = b[actual_arrival_to] - b[actual_departure_from]

# Create A Graph

In [13]:
# Creating a pair of gps locations and calculating the signal at that time instance:
# temp['Road_Segment_from'] = list(zip(temp.GPS_Latitude_x, temp.GPS_Longitude_x))
# temp['Road_Segment_to'] = list(zip(temp.GPS_Latitude_y, temp.GPS_Longitude_y))
# temp

# aggregating the gps locations of the same punkts
df_new_x = df_1.groupby(['halt_id_x'], as_index=False)['GPS_Latitude_x', 'GPS_Longitude_x'].mean()
df_new_y = df_1.groupby(['halt_id_y'], as_index=False)['GPS_Latitude_y', 'GPS_Longitude_y'].mean()
# Merging the two frames signifying a road segment
final = df_1.merge(df_new_x, on=['halt_id_x'])
final = final.merge(df_new_y, on=['halt_id_y'])


In [14]:
final.rename(columns={'GPS_Latitude_x_y':'GPS_LAN_STELLE_FROM', 'GPS_Longitude_x_y':'GPS_LON_STELLE_FROM',
                      'GPS_Latitude_y_y':'GPS_LAN_STELLE_TO', 'GPS_Longitude_y_y':'GPS_LON_STELLE_TO'}, inplace=True)
# final

In [16]:
final

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x_x,GPS_Longitude_x_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y_x,GPS_Longitude_y_x,GPS_Bearing_y,halt_punkt_ist_aktiv_y,halt_id,halt_diva_x,halt_kurz_x,halt_lang_x,halt_ist_aktiv_x,halt_diva_y,halt_kurz_y,halt_lang_y,halt_ist_aktiv_y,target_stationary,target_travel_time,delay,actual_stationary,actual_travel_time,GPS_LAN_STELLE_FROM,GPS_LON_STELLE_FROM,GPS_LAN_STELLE_TO,GPS_LON_STELLE_TO
0,918,1,06.01.16,11652,1,2,3744,50,ZENK,06.01.16,65880,65906,65898,65906,3,2043,50,RKIK,06.01.16,65988,65997,66000,65997,75165,5091,1,1,1,BKUE - GART,112395,2663,2796,6081,6661,6661,50,2663,47.318230,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,18,90,26,0,91,47.318319,8.581964,47.316607,8.583933
1,918,1,06.01.16,11664,2,2,3744,50,ZENK,06.01.16,24480,24561,24492,24572,3,2043,50,RKIK,06.01.16,24576,24657,24588,24657,75179,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.318230,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,81,11,85,47.318319,8.581964,47.316607,8.583933
2,918,1,09.01.16,11655,2,2,3744,50,ZENK,09.01.16,53280,53313,53292,53325,3,2043,50,RKIK,09.01.16,53376,53407,53388,53407,139140,5091,1,1,1,BKUE - GART,113211,2663,2796,6081,6661,6661,50,2663,47.318230,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,33,12,82,47.318319,8.581964,47.316607,8.583933
3,918,1,08.01.16,11653,2,2,3744,50,ZENK,08.01.16,24480,24510,24492,24510,3,2043,50,RKIK,08.01.16,24576,24586,24588,24586,75179,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.318230,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,30,0,76,47.318319,8.581964,47.316607,8.583933
4,918,1,08.01.16,11646,2,2,3744,50,ZENK,08.01.16,53280,53319,53292,53334,3,2043,50,RKIK,08.01.16,53376,53434,53388,53445,75211,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.318230,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,39,15,100,47.318319,8.581964,47.316607,8.583933
5,918,1,08.01.16,11653,2,2,3744,50,ZENK,08.01.16,42480,42497,42492,42508,3,2043,50,RKIK,08.01.16,42576,42591,42588,42591,75199,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.318230,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,12,84,17,11,83,47.318319,8.581964,47.316607,8.583933
6,918,1,07.01.16,11646,2,2,3744,50,ZENK,07.01.16,60480,60510,60498,60519,3,2043,50,RKIK,07.01.16,60588,60628,60600,60628,75219,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.318230,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Reform. Kirche",True,18,90,30,9,109,47.318319,8.581964,47.316607,8.583933
7,918,1,08.01.16,11653,2,2,3744,50,ZENK,08.01.16,28080,28093,28092,28104,3,2043,50,RKIK,08.01.16,28176,28188,28188,28188,75183,5091,1,1,1,BKUE - GART,112361,2663,2796,6081,6661,6661,50,2663,47.318230,8.581693,77.0,True,50,2796,47.316607,8.583933,189.0,True,2796,3744,ZENK,"Küsnacht ZH, Zentrum",True,2043,RKIK,"Küsnacht ZH, Refo

In [17]:
final['Road_Segment_From'] = list(zip(final.GPS_Latitude_x_x, final.GPS_Longitude_x_x))
final['Road_Segment_To'] = list(zip(final.GPS_Latitude_y_x, final.GPS_Longitude_y_x))
final['Road_Segment_Node'] = list(zip(final.Road_Segment_From, final.Road_Segment_To))
final['Route_Node_id_tuple'] = list(zip(final.halt_punkt_id_von, final.halt_punkt_id_nach))


###
# final['Road_Segment_From'] = list(zip(final.GPS_LAN_STELLE_FROM, final.GPS_LON_STELLE_FROM))
# final['Road_Segment_To'] = list(zip(final.GPS_LAN_STELLE_TO, final.GPS_LON_STELLE_TO))
# final['Road_Segment_Node'] = list(zip(final.Road_Segment_From, final.Road_Segment_To))
# final['Route_Node_id_tuple'] = list(zip(final.halt_id_x, final.halt_id_y))
# # final

In [ ]:
# columns_to_drop = ['halt_diva_von', 'halt_punkt_diva_von', 'halt_diva_nach', 'halt_punkt_diva_nach', 'fw_no', 'fw_typ', 'fw_kurz']
# final.drop(columns=columns_to_drop, inplace=True)

# Construct the graph:

    Dealing with granularity issues:

In [18]:
d = pd.to_datetime(final['betriebsdatum'], format='%d.%m.%y', dayfirst=True)
# Monday == 0 … Sunday == 6
final['day_of_week'] = [i.weekday() for i in d]

In [19]:
final = final[final.fw_typ != 2]

In [20]:
final.fw_no.value_counts()[:5]

1    1196404
2    1149323
4     130509
3      99602
6      27650
Name: fw_no, dtype: int64

In [21]:
list(set(zip(final.GPS_Latitude_x_x, final.GPS_Longitude_x_x)))[:5]

[(47.30712, 8.548644),
 (47.409068, 8.595832),
 (47.38303, 8.499871),
 (47.358689, 8.527014),
 (47.370287, 8.514156)]

In [22]:
pd.options.display.max_columns = 100
final.fw_typ.value_counts()

1    2724077
3       2750
Name: fw_typ, dtype: int64

In [23]:
# # encode routes
final['Route_Node_id_str'] = final.Route_Node_id_tuple.apply(lambda x: '{}_{}'.format(x[0], x[1]))
le = preprocessing.LabelEncoder()
le.fit(final.Route_Node_id_str)
final['Route_Node_id'] = le.transform(final.Route_Node_id_str)

C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [24]:
final = final[((final.day_of_week != 5) & (final.day_of_week != 6))]
final = final[(final['soll_an_von'] >= 21600) & (final['soll_an_von'] <= 72000)]

In [25]:
final.fw_no.value_counts()[:5]

1    759498
2    725564
4     87933
3     61101
6     17487
Name: fw_no, dtype: int64

In [26]:
len(set(final.Road_Segment_Node))

1824

In [27]:
final['actual_travel_time_n_target'] = final.actual_travel_time.div(final.target_travel_time, axis=0)
final['actual_travel_time_n_target_mean'] = final.actual_travel_time.div(final.target_travel_time.mean(), axis=0)

In [28]:
time_period = 600
# Previously was at 5 am
starting_time = 21600
# indicates 10 mins intervals
# Number of bins:
n_bins = int(np.floor((72000 - starting_time)/time_period))
labels = [i for i in range(n_bins)]
# Finding the correct bins:
list_of_bins = [starting_time + time_period * i for i in range(n_bins + 1)]
final['intervals'] = pd.cut(final[target_arrival_from], bins=list_of_bins, retbins=False, labels=labels, right=True, include_lowest=True)

In [29]:
len(labels)

84

In [30]:
final.Route_Node_id.value_counts()[:5]

345     4564
428     4563
1280    4423
1378    4399
1210    4079
Name: Route_Node_id, dtype: int64

In [31]:
# final['actual_travel_time_n_target'] = final.actual_travel_time / final.target_travel_time
# final.actual_travel_time_n_target = final.actual_travel_time_n_target - final.actual_travel_time_n_target.mean()
# final.actual_travel_time_n_target_mean = final.actual_travel_time_n_target_mean - final.actual_travel_time_n_target_mean.mean()

In [ ]:
# %matplotlib
# a = final[final.Route_Node_id == 1158    ]
# a = a[a.betriebsdatum == '05.01.16']
# # a = a.sort_values(by=target_departure_from)
# a = a.groupby(by='intervals', as_index=False).mean()
# # a.intervals.value_counts()
# # a = a.groupby(by='intervals', as_index=False).mean()
# # a.fillna(value=0)
# # plt.plot(a[target_departure_from], a.target_travel_time)
# plt.plot(a.intervals.values.tolist(), a.actual_travel_time_n_target.values.tolist(), a.intervals.values.tolist(), a.actual_travel_time_n_target_mean.values.tolist())

In [ ]:
# a = final[final.Route_Node_id == 1158]
# a = a[a.betriebsdatum == '05.01.16']
# for i in set(a.fahrweg_id):
#     b = a[a.fahrweg_id == i]
#     b.sort_values(by=['intervals', target_departure_from], inplace=True)
#     if b.shape[0] < 50:
#         continue
#     plt.plot(b[target_departure_from], b.actual_travel_time_n_target)
# a = a.groupby(by='intervals', as_index=False).mean()
# plt.title('Signal')
# plt.xlabel('Times of the day')
# plt.ylabel('')
# a.sort_values(by=['intervals', target_departure_from], inplace=True)
# plt.scatter([time_period * i + 21600 for i in a.intervals.values.tolist()], np.array(a.actual_travel_time_n_target.values.tolist()), marker='^', s=1000)

In [32]:
final.Route_Node_id.value_counts()[580:600]

1276    1118
1737    1118
1817    1118
330     1118
41      1118
1014    1118
502     1118
1670    1117
1802    1117
531     1117
1807    1117
1361    1117
1709    1117
255     1117
1584    1117
1       1117
1217    1117
244     1117
1819    1117
29      1117
Name: Route_Node_id, dtype: int64

In [33]:
# Find the Nodes with gran. problems:

list_road_seg = final.Route_Node_id.value_counts()
list_road_seg_id = []
list_empty_bins_index = []
list_to_drop = []

for index, item in list_road_seg.items():
    temp = final[final.Route_Node_id == index]
    if (0 in (temp.intervals.value_counts().value_counts())):
        list_road_seg_id.append(index)
        list_empty_bins_index.append(temp.intervals.value_counts().value_counts()[0])
        list_to_drop.append(index)
    else:
        list_road_seg_id.append(index)
        list_empty_bins_index.append(0)
list_empty_bins_index = np.array(list_empty_bins_index)

In [34]:
len(set(final.Route_Node_id))

1824

In [35]:
%matplotlib
len(list_empty_bins_index)
pd.Series(list_empty_bins_index).hist()
plt.title('Distribution of empty and non empty bins - 10 mins - Start at 7 - 10')
plt.xlabel('Road ID')
plt.ylabel('Number of empty bins')

Using matplotlib backend: Qt5Agg


Text(0, 0.5, 'Number of empty bins')

In [36]:
%matplotlib
# Plot the halts vs. empty bins
plt.scatter(list_road_seg_id, list_empty_bins_index)
plt.title('Scatter of empty and non empty bins - 10 mins  - Start at 7 - 10')
plt.xlabel('Road ID')
plt.ylabel('Number of empty bins')


Using matplotlib backend: Qt5Agg


Text(0, 0.5, 'Number of empty bins')

In [38]:
print(np.count_nonzero(list_empty_bins_index < 6))
print(np.count_nonzero(list_empty_bins_index == 0))

898
788


# A Clear Issue with granularity
solving it by limiting the start date and visualizing the distribution


In [39]:
# rev_dist_bins = pd.Series(index = final.intervals.value_counts().values, data=final.intervals.value_counts().index.values)
plt.scatter(final.intervals.value_counts().index.values, final.intervals.value_counts().values)
plt.title('scatter - bin number vs data count - start at 7, end at 12 - 10 mins')
plt.xlabel('Bin number')
plt.ylabel('Count of Data points')

Text(0, 0.5, 'Count of Data points')

In [43]:
final.intervals.value_counts()[:5]

10    23156
7     23078
6     22900
11    22834
9     22791
8     22787
70    22765
64    22740
69    22709
67    22692
5     22685
12    22683
13    22674
66    22642
72    22629
73    22536
71    22533
68    22500
65    22498
63    22326
14    22131
75    22116
4     22095
74    21973
62    21866
76    21841
16    21583
61    21523
15    21506
77    21382
      ...  
82    19074
21    19055
35    19023
33    19012
46    18986
83    18949
20    18906
81    18861
54    18851
27    18848
24    18825
30    18825
48    18810
51    18809
47    18798
28    18791
22    18785
56    18784
31    18781
29    18757
53    18750
23    18749
1     18742
32    18709
25    18704
55    18694
52    18692
26    18680
50    18668
49    18619
Name: intervals, Length: 83, dtype: int64

In [44]:
np.array(final.intervals.value_counts().index.values)

array([10,  7,  6, 11,  9,  8, 70, 64, 69, 67,  5, 12, 13, 66, 72, 73, 71,
       68, 65, 63, 14, 75,  4, 74, 62, 76, 16, 61, 15, 77,  3, 60, 78, 17,
       59, 79,  2, 18, 58, 40, 39, 42, 37, 19, 43, 80, 36, 57, 45, 38, 41,
       44, 34, 82, 21, 35, 33, 46, 83, 20, 81, 54, 27, 24, 30, 48, 51, 47,
       28, 22, 56, 31, 29, 53, 23,  1, 32, 25, 55, 52, 26, 50, 49,  0],
      dtype=int64)

In [45]:
# Drop those datapoints:
print(np.count_nonzero(list_empty_bins_index != 0))

1036


In [46]:
set(list_empty_bins_index)

{0,
 1,
 2,
 3,
 4,
 6,
 8,
 9,
 10,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 68,
 70,
 71,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83}

In [47]:
final = final[~final.Route_Node_id.isin(list_to_drop)]

In [48]:
len(final.Road_Segment_Node)

1184871

In [49]:
final.Route_Node_id.value_counts()

345     4564
428     4563
1280    4423
1378    4399
1210    4079
320     3559
1286    3544
985     3467
176     3465
803     3458
429     3441
169     3431
801     3421
600     3380
1069    3364
343     3357
262     3320
645     3312
635     3311
175     3304
641     3304
644     3304
630     3303
634     3300
631     3296
629     3294
633     3293
217     3293
640     3293
1396    3291
        ... 
1394    1078
798     1072
384     1071
992     1071
1755    1071
115     1071
670     1070
993     1070
714     1070
713     1069
969     1069
661     1068
1859    1063
651     1062
464     1060
1650     840
1393     840
1259     840
967      840
1798     840
1223     840
1348     840
1492     839
1260     839
1349     839
1744     839
1651     839
1222     838
289      837
148      837
Name: Route_Node_id, Length: 788, dtype: int64

# Define new signal: Normalized travel time - close to TTI - Version 1, normalize by target travel time

In [50]:
final['actual_travel_time_n_target'] = final.actual_travel_time.div(final.target_travel_time, axis=0)
final

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x_x,GPS_Longitude_x_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y_x,GPS_Longitude_y_x,GPS_Bearing_y,halt_punkt_ist_aktiv_y,halt_id,halt_diva_x,halt_kurz_x,halt_lang_x,halt_ist_aktiv_x,halt_diva_y,halt_kurz_y,halt_lang_y,halt_ist_aktiv_y,target_stationary,target_travel_time,delay,actual_stationary,actual_travel_time,GPS_LAN_STELLE_FROM,GPS_LON_STELLE_FROM,GPS_LAN_STELLE_TO,GPS_LON_STELLE_TO,Road_Segment_From,Road_Segment_To,Road_Segment_Node,Route_Node_id_tuple,day_of_week,Route_Node_id_str,Route_Node_id,actual_travel_time_n_target,actual_travel_time_n_target_mean,intervals
14342,2,1,07.01.16,2079,6,11,1357,0,KALK,07.01.16,56856,56883,56880,56908,12,478,0,BEZI,07.01.16,56946,56970,56964,56986,44449,18069,1,1,1,FARB - BTIE,112467,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,27,25,62,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",3,4292_2829,361,0.939394,0.934367,58
14343,3,1,07.01.16,3051,7,9,1357,0,KALK,07.01.16,43620,43624,43644,43643,10,478,0,BEZI,07.01.16,43710,43702,43728,43724,44802,21368,1,1,1,ALBR - KLUS,112472,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,4,19,59,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",3,4292_2829,361,0.893939,0.889155,36
14344,2,1,04.01.16,2070,8,11,1357,0,KALK,04.01.16,57756,57806,57780,57825,12,478,0,BEZI,04.01.16,57846,57889,57864,57902,44515,18069,1,1,1,FARB - BTIE,114432,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,50,19,64,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",0,4292_2829,361,0.969697,0.964507,60
14345,3,1,07.01.16,3015,3,9,1357,0,KALK,07.01.16,55320,55307,55344,55329,10,478,0,BEZI,07.01.16,55410,55393,55428,55407,44680,21368,1,1,1,ALBR - KLUS,113791,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,-13,22,64,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",3,4292_2829,361,0.969697,0.964507,56
14346,3,1,07.01.16,3051,7,9,1357,0,KALK,07.01.16,70620,70625,70644,70642,10,478,0,BEZI,07.01.16,70710,70705,70728,70724,44814,21368,1,1,1,ALBR - KLUS,112472,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,5,17,63,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",3,4292_2829,361,0.954545,0.949437,81
14347,2,1,04.01.16,2069,4,11,1357,0,KALK,04.01.16,42456,42471,42480,42492,12,478,0,BEZI,04.01.16,42546,42558,42564,42585,44329,18069,1,1,1,FARB - BTIE,113308,1906,130

# Define new signal: Normalized travel time - Version 2, normalize by mean of target in that interval

In [51]:
final['actual_travel_time_n_target_mean'] = final.actual_travel_time.div(final.target_travel_time.mean(), axis=0)
final

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach,halt_punkt_id,halt_punkt_diva_x,halt_id_x,GPS_Latitude_x_x,GPS_Longitude_x_x,GPS_Bearing_x,halt_punkt_ist_aktiv_x,halt_punkt_diva_y,halt_id_y,GPS_Latitude_y_x,GPS_Longitude_y_x,GPS_Bearing_y,halt_punkt_ist_aktiv_y,halt_id,halt_diva_x,halt_kurz_x,halt_lang_x,halt_ist_aktiv_x,halt_diva_y,halt_kurz_y,halt_lang_y,halt_ist_aktiv_y,target_stationary,target_travel_time,delay,actual_stationary,actual_travel_time,GPS_LAN_STELLE_FROM,GPS_LON_STELLE_FROM,GPS_LAN_STELLE_TO,GPS_LON_STELLE_TO,Road_Segment_From,Road_Segment_To,Road_Segment_Node,Route_Node_id_tuple,day_of_week,Route_Node_id_str,Route_Node_id,actual_travel_time_n_target,actual_travel_time_n_target_mean,intervals
14342,2,1,07.01.16,2079,6,11,1357,0,KALK,07.01.16,56856,56883,56880,56908,12,478,0,BEZI,07.01.16,56946,56970,56964,56986,44449,18069,1,1,1,FARB - BTIE,112467,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,27,25,62,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",3,4292_2829,361,0.939394,0.938609,58
14343,3,1,07.01.16,3051,7,9,1357,0,KALK,07.01.16,43620,43624,43644,43643,10,478,0,BEZI,07.01.16,43710,43702,43728,43724,44802,21368,1,1,1,ALBR - KLUS,112472,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,4,19,59,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",3,4292_2829,361,0.893939,0.893192,36
14344,2,1,04.01.16,2070,8,11,1357,0,KALK,04.01.16,57756,57806,57780,57825,12,478,0,BEZI,04.01.16,57846,57889,57864,57902,44515,18069,1,1,1,FARB - BTIE,114432,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,50,19,64,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",0,4292_2829,361,0.969697,0.968887,60
14345,3,1,07.01.16,3015,3,9,1357,0,KALK,07.01.16,55320,55307,55344,55329,10,478,0,BEZI,07.01.16,55410,55393,55428,55407,44680,21368,1,1,1,ALBR - KLUS,113791,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,-13,22,64,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",3,4292_2829,361,0.969697,0.968887,56
14346,3,1,07.01.16,3051,7,9,1357,0,KALK,07.01.16,70620,70625,70644,70642,10,478,0,BEZI,07.01.16,70710,70705,70728,70724,44814,21368,1,1,1,ALBR - KLUS,112472,1906,1306,4292,2829,2829,0,1906,47.374529,8.521273,100.0,False,0,1306,47.373946,8.525869,119.0,False,1306,1357,KALK,"Zürich,Kalkbreite/Bhf.Wiedikon",True,478,BEZI,"Zürich, Bezirksgebäude",True,24,66,5,17,63,47.374475,8.521106,47.374076,8.525594,"(47.374529, 8.521273)","(47.373946, 8.525869)","((47.374529, 8.521273), (47.373946, 8.525869))","(4292, 2829)",3,4292_2829,361,0.954545,0.953748,81
14347,2,1,04.01.16,2069,4,11,1357,0,KALK,04.01.16,42456,42471,42480,42492,12,478,0,BEZI,04.01.16,42546,42558,42564,42585,44329,18069,1,1,1,FARB - BTIE,113308,1906,130

# Create Signal First!

In [52]:
# Creating Signals for all of the nodes:
List_of_Nodes = [str(i) for i in list(set(final.Route_Node_id))]
List_of_Nodes

['17',
 '18',
 '19',
 '21',
 '22',
 '23',
 '24',
 '29',
 '31',
 '38',
 '40',
 '41',
 '43',
 '44',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '57',
 '58',
 '59',
 '60',
 '63',
 '64',
 '65',
 '69',
 '72',
 '75',
 '77',
 '78',
 '88',
 '89',
 '90',
 '91',
 '92',
 '104',
 '105',
 '107',
 '108',
 '110',
 '111',
 '115',
 '116',
 '119',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '134',
 '143',
 '147',
 '148',
 '162',
 '163',
 '165',
 '169',
 '170',
 '172',
 '173',
 '174',
 '175',
 '176',
 '178',
 '179',
 '184',
 '186',
 '194',
 '196',
 '197',
 '198',
 '199',
 '200',
 '202',
 '205',
 '207',
 '209',
 '210',
 '212',
 '213',
 '214',
 '217',
 '218',
 '219',
 '220',
 '221',
 '224',
 '225',
 '232',
 '234',
 '241',
 '245',
 '247',
 '248',
 '249',
 '250',
 '251',
 '252',
 '255',
 '259',
 '260',
 '261',
 '262',
 '263',
 '264',
 '268',
 '271',
 '274',
 '275',
 '276',
 '279',
 '280',
 '289',
 '292',
 '293',
 '295',
 '299',
 '301',


In [53]:
len(List_of_Nodes)

788

In [54]:
List_of_head_data = {}
for i in List_of_Nodes:
    temp_df = final[final.Route_Node_id == int(i)]
#     temp_df = temp_df.sort_values(by=['betriebsdatum'])
    time_series_for_node = []
    for j in sorted(list(set(temp_df.betriebsdatum)), key=lambda x: (x[3], x[4], x[0], x[1])):
#         print(j)
        time_series = temp_df[temp_df.betriebsdatum == j]
        time_series = time_series.groupby(by='intervals', as_index=False).mean()
        time_series = time_series['actual_travel_time_n_target']
        time_series_for_node.append(time_series)
    final_series = pd.concat(time_series_for_node, ignore_index=True)
#     if len(final_series) < 2100:
#         print(i)
    List_of_head_data[i] = final_series
    #     print(final_series)
# data_df

In [55]:
# a = List_of_head_data['184']
new_dict = {}
for key, value in List_of_head_data.items():
    new_dict[key] = value.isna().sum()

In [56]:
new_dict

{'17': 6,
 '18': 0,
 '19': 0,
 '21': 0,
 '22': 0,
 '23': 0,
 '24': 3,
 '29': 3,
 '31': 1,
 '38': 4,
 '40': 10,
 '41': 0,
 '43': 0,
 '44': 0,
 '46': 17,
 '47': 14,
 '48': 0,
 '49': 1,
 '50': 4,
 '51': 23,
 '52': 1,
 '53': 21,
 '54': 3,
 '57': 4,
 '58': 1,
 '59': 0,
 '60': 3,
 '63': 4,
 '64': 2,
 '65': 3,
 '69': 1,
 '72': 0,
 '75': 0,
 '77': 2,
 '78': 0,
 '88': 0,
 '89': 2,
 '90': 4,
 '91': 2,
 '92': 0,
 '104': 0,
 '105': 4,
 '107': 2,
 '108': 4,
 '110': 6,
 '111': 7,
 '115': 32,
 '116': 20,
 '119': 3,
 '121': 1,
 '122': 0,
 '123': 0,
 '124': 0,
 '125': 0,
 '126': 29,
 '127': 24,
 '128': 0,
 '129': 0,
 '130': 0,
 '131': 0,
 '132': 4,
 '134': 25,
 '143': 22,
 '147': 35,
 '148': 3,
 '162': 2,
 '163': 3,
 '165': 4,
 '169': 1,
 '170': 8,
 '172': 1,
 '173': 0,
 '174': 1,
 '175': 0,
 '176': 3,
 '178': 0,
 '179': 0,
 '184': 0,
 '186': 0,
 '194': 4,
 '196': 0,
 '197': 0,
 '198': 4,
 '199': 0,
 '200': 9,
 '202': 0,
 '205': 1,
 '207': 11,
 '209': 0,
 '210': 0,
 '212': 0,
 '213': 12,
 '214': 0,
 '2

In [57]:
# Finding dicts with distribution
nan_dist = pd.Series(new_dict)
nan_dist.hist()

In [ ]:
# Find which indecies have above 10 missing values

In [70]:
# print(len(final_data_df.columns))
final_data_df = pd.DataFrame.from_dict(List_of_head_data)
print(sorted(final_data_df.isna().sum()))
final_data_df.dropna(thresh=len(final_data_df) - 15, axis=1, inplace=True)
print(len(final_data_df.columns))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [71]:
print(sorted(final_data_df.isna().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [74]:
final_data_df.isna().sum().sum()

0

In [76]:
final_data_df

,17,18,19,21,22,23,24,29,31,38,40,41,43,44,47,48,49,50,52,54,57,58,59,60,63,64,65,69,72,75,77,78,88,89,90,91,92,104,105,107,108,110,111,119,121,122,123,124,125,128,...,1708,1709,1727,1728,1731,1732,1733,1735,1737,1739,1740,1744,1753,1757,1769,1782,1783,1784,1785,1786,1791,1792,1797,1798,1799,1800,1801,1802,1805,1807,1808,1809,1810,1811,1812,1815,1817,1818,1819,1827,1828,1829,1830,1832,1847,1849,1851,1852,1854,1860
0,1.074561,0.974116,1.182540,1.027778,1.092593,1.283333,1.100000,1.152778,0.907407,1.166667,1.150000,1.083333,1.016667,1.064103,1.031250,1.041667,1.089744,1.015152,1.016129,1.106061,0.500000,0.759259,0.883838,1.015625,0.904762,0.981481,1.100000,0.916667,1.071429,0.974359,1.358333,0.972222,1.045455,0.981481,1.222222,1.097222,1.055556,1.095238,0.939394,0.916667,1.048611,1.015152,1.133333,1.055556,1.194444,1.069444,1.500000,0.925439,1.085470,1.203704,...,0.935897,1.104167,1.159091,0.850000,1.049020,1.044444,1.050926,1.066667,1.089744,0.903846,1.000000,1.380952,0.947917,1.023810,1.089744,1.187500,1.078125,1.302083,0.992063,1.737179,1.266667,0.994048,1.073529,0.642857,0.979167,1.203704,0.888889,1.000000,1.100000,1.150000,1.030303,0.944444,0.966667,1.000000,1.018519,0.976190,1.018519,0.961538,0.738095,0.961538,1.011905,1.108333,0.666667,1.022222,1.187500,1.077778,1.005952,0.850000,1.045752,1.066667
1,1.037281,1.018519,1.293651,0.916667,1.222222,1.133333,1.116667,1.027778,0.962963,0.977778,1.100000,1.178571,0.975000,1.307692,1.005208,0.909722,0.952991,0.992424,0.994624,1.075758,0.777778,0.851852,0.954545,1.145833,0.857143,0.981481,1.150000,0.875000,1.119048,1.262108,1.333333,0.967593,1.181818,0.990741,1.259259,1.083333,0.965278,1.166667,0.946970,1.000000,1.027778,1.075758,1.200000,1.069444,1.166667,1.333333,0.914141,0.905702,1.256410,1.074074,...,1.243590,1.416667,1.045455,0.825000,0.872549,1.211111,0.979167,0.933333,1.070513,0.961538,1.000000,1.380952,0.986111,1.011905,1.064103,1.000000,1.020833,1.250000,1.099206,1.634615,1.100000,1.000000,1.046569,0.571429,1.104167,1.259259,0.740741,1.033333,0.483333,1.116667,0.946970,0.981481,1.066667,1.179487,1.370370,1.083333,0.962963,1.083333,1.083333,0.961538,0.964286,1.050000,0.911111,0.950000,1.152778,1.083333,1.130952,1.166667,1.062092,0.955556
2,1.000000,1.116162,1.214286,0.972222,1.203704,1.191667,1.150000,1.062500,1.268519,0.847222,1.100000,1.023810,1.000000,1.243590,1.145833,0.937500,0.948718,1.174242,1.083333,0.984848,0.666667,1.129630,0.969697,1.031250,1.273810,1.055556,1.083333,1.027778,1.035714,1.107550,1.483333,0.972222,1.166667,1.041667,1.250000,1.097222,1.361111,1.077381,1.000000,1.047619,1.043981,1.166667,1.133333,1.097222,1.138889,0.939815,1.030303,0.978070,1.397436,1.120370,...,0.935897,1.281250,0.992424,0.850000,1.039216,1.266667,1.018519,1.116667,1.102564,1.012821,0.930556,1.190476,0.965278,1.013605,1.038462,1.166667,1.041667,1.041667,1.039683,1.649573,1.088889,0.976190,1.019608,0.690476,1.208333,1.259259,0.851852,1.116667,1.283333,1.316667,1.030303,0.953704,0.933333,1.282051,0.981481,1.023810,1.018519,1.064103,0.976190,1.179487,1.083333,1.116667,1.566667,1.025926,1.333333,1.050000,1.321429,0.983333,1.078431,0.996296
3,0.991228,1.031566,1.190476,1.020833,1.055556,1.133333,1.125000,1.069444,1.000000,1.000000,1.150000,1.166667,0.927778,1.034188,1.062500,0.854167,1.012821,1.121212,1.059140,1.121212,0.611111,0.916667,0.858586,0.945216,1.047619,0.981481,0.950000,1.013889,1.035714,1.373219,1.341667,0.916667,1.083333,1.018519,1.212963,1.090278,1.004630,1.011905,1.090909,0.988095,1.060185,1.037879,1.179630,0.958333,1.152778,1.300926,0.924242,0.899123,1.256410,1.216049,...,0.955556,1.479167,0.909091,0.783333,0.931373,1.244444,1.046296,1.041667,0.980769,1.384615,1.055556,1.119048,1.027778,1.065476,1.025641,1.041667,1.250000,1.156250,1.044974,1.634615,0.888889,1.035714,0.950980,0.714286,0.895833,1.000000,1.111111,0.933333,1.041667,1.016667,1.212121,1.018519,0.933333,1.044872,0.944444,1.023810,0.981481,0.980769,1.083333,1.076923,1.017857,0.933333,0.872222,1.034722,1.138889,1.1

In [73]:
final_data_df = final_data_df.interpolate()

In [82]:
# 726     656
# 769     581
sig_1 = final_data_df['17']
sig_2 = final_data_df['22']

In [83]:
plt.plot(final_data_df.index, sig_1, final_data_df.index, sig_2)

In [84]:
np.corrcoef(sig_1,sig_2)

array([[ 1.        , -0.04909136],
       [-0.04909136,  1.        ]])

In [88]:
# # 459 is corr!
# sig_1 = final_data_df['17']
# # 619! 607!
# list_of_node_crosscoef = []
# for i in edge_series[10]:
#     sig_2 = final_data_df[str(i)]
# #     sig2 = sig2[sig2.betriebsdatum == '11.01.16']
# #     sig2.sort_values(by='soll_an_von', inplace=True)
# #     sig2 = sig2.groupby(by='intervals', as_index=False).mean()
# #     sig2_y = np.array(sig2.actual_travel_time_n_target)
#     list_of_node_crosscoef.append((i, np.corrcoef(sig_1, sig_2)[0][1]))

NameError: name 'edge_series' is not defined

In [86]:
# zlst = list(zip(*list_of_node_crosscoef))
# list_of_node_crosscoef = pd.Series(zlst[1], index = zlst[0])
# list_of_node_crosscoef = list_of_node_crosscoef.dropna().sort_values()
# plt.scatter(list_of_node_crosscoef.index.values.tolist(), list_of_node_crosscoef.values.tolist())

In [ ]:
# edge_series.value_counts()

In [ ]:
# edge_series[edge_series.index == 730]

In [ ]:
# final[final.Route_Node_id == 693]
# 607 - 	(47.37303733083076, 8.528243450630427)
# 619 (47.37298538959543, 8.542866657639465)
# 693 (47.37871448213164, 8.548137124824857)

In [ ]:
# nx.draw_networkx(G)

# Signal is Defined, Create the graphs in two ways! Using the signal DF!

# Create The graph - V1

In [89]:
final['Mid_point_GPS_data'] = final.Road_Segment_Node.apply(lambda x: (((x[0][0] + x[1][0])/2) , ((x[0][1] + x[1][1])/2)))

In [90]:
len(final_data_df.columns.values.tolist())

655

In [91]:
list_of_nodes_Road = list(set(final_data_df.columns.values.tolist()))
# list_of_nodes = list(set(zip(final.Route_Node_id, final.Mid_point_GPS_data)))
(list_of_nodes_Road)

['1407',
 '1423',
 '1070',
 '1447',
 '1066',
 '619',
 '1219',
 '532',
 '388',
 '1406',
 '517',
 '608',
 '616',
 '1380',
 '31',
 '1498',
 '1492',
 '430',
 '1589',
 '495',
 '1520',
 '1236',
 '596',
 '1080',
 '330',
 '1272',
 '1682',
 '1280',
 '1169',
 '1860',
 '631',
 '64',
 '544',
 '210',
 '1354',
 '1109',
 '1124',
 '1428',
 '292',
 '350',
 '1053',
 '59',
 '1349',
 '834',
 '1103',
 '361',
 '1025',
 '1436',
 '455',
 '1832',
 '1563',
 '1309',
 '882',
 '429',
 '1207',
 '1226',
 '60',
 '69',
 '1122',
 '130',
 '1646',
 '1575',
 '57',
 '1119',
 '1791',
 '224',
 '186',
 '1662',
 '919',
 '910',
 '1513',
 '1573',
 '598',
 '1382',
 '1786',
 '1588',
 '247',
 '354',
 '483',
 '374',
 '1451',
 '883',
 '264',
 '1286',
 '889',
 '49',
 '252',
 '107',
 '88',
 '1547',
 '638',
 '1623',
 '214',
 '1426',
 '1020',
 '52',
 '251',
 '575',
 '590',
 '1535',
 '58',
 '1663',
 '1223',
 '594',
 '90',
 '165',
 '639',
 '359',
 '1614',
 '1400',
 '169',
 '1273',
 '920',
 '507',
 '1469',
 '804',
 '212',
 '1670',
 '307',
 

In [92]:
from geopy.distance import vincenty
from geopy.distance import geodesic
from networkx.drawing.nx_pydot import write_dot

In [93]:
list_of_nodes_Road_toKeep = [int(i) for i in list_of_nodes_Road]
final = final[final.Route_Node_id.isin(list_of_nodes_Road_toKeep)]
# final

In [94]:
# Create List of dicts:


new_final = final.groupby(by='Route_Node_id', as_index=False).min()


In [95]:
Dict_List = pd.Series(new_final.Mid_point_GPS_data.values,index=new_final.Route_Node_id).to_dict()
Dict_List

{17: (47.4458855, 8.566161000000001),
 18: (47.3701085, 8.5168865),
 19: (47.370029, 8.5114495),
 21: (47.369026500000004, 8.5123095),
 22: (47.3709435, 8.5151685),
 23: (47.439938, 8.565883),
 24: (47.436366, 8.5599745),
 29: (47.3683275, 8.4984595),
 31: (47.3691805, 8.504461500000001),
 38: (47.368728000000004, 8.493496),
 40: (47.3688785, 8.4938035),
 41: (47.368360499999994, 8.498751500000001),
 43: (47.3744365, 8.5230255),
 44: (47.373768, 8.528082999999999),
 47: (47.375869, 8.550704),
 48: (47.379073000000005, 8.5482145),
 49: (47.37817, 8.5462025),
 50: (47.4003815, 8.534158),
 52: (47.3981905, 8.537478499999999),
 54: (47.400353, 8.5342835),
 57: (47.3983765, 8.5330635),
 58: (47.39811, 8.5308595),
 59: (47.396533, 8.534927),
 60: (47.3970405, 8.532651999999999),
 63: (47.38981750000001, 8.4810385),
 64: (47.391331, 8.481578500000001),
 65: (47.3919405, 8.476981),
 69: (47.408739, 8.5477125),
 72: (47.4117915, 8.545974000000001),
 75: (47.4107655, 8.545522),
 77: (47.411602, 

In [96]:
# Find edges and weights:
list_of_edges = []

# iterate over each mid point aka road segment
for i in list_of_nodes_Road:
    # Look at every other road segment, if physical distance was less than 2.5 kilometers, add an edge
    
    for key, value in Dict_List.items():
        if Dict_List[int(i)] != value:
            if vincenty(Dict_List[int(i)], value).meters < 1500:
                list_of_edges.append((int(i), key))
    
#     a = new_final[new_final.Route_Node_id == int(i)]
#     a.Route_Node_id_tuple.iloc[5]
#     for j in list_of_nodes:
#         if i[0] != j[0]:
#             if vincenty(i[1], j[1]).meters < 1000:
#                 list_of_edges.append((i[0], j[0]))

C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  # Remove the CWD from sys.path while we load stuff.


In [97]:
zlst = list(zip(*list_of_edges))
edge_series = pd.Series(zlst[1], index = zlst[0])
edge_series.value_counts()

381     178
217     178
213     177
386     177
218     177
1784    177
207     176
631     176
1851    176
210     176
343     176
219     175
212     175
205     175
641     174
344     174
646     174
221     173
214     173
209     173
295     173
633     173
600     173
512     173
599     172
1785    172
650     171
513     171
388     170
1860    170
       ... 
1492     13
1259     13
1348     13
1744     13
24       13
843      12
1650     12
1349     12
1651     12
1223     12
1460     11
967      10
1222     10
1277     10
289      10
23       10
88       10
875      10
410       9
1686      9
90        9
982       8
148       8
276       7
194       6
447       6
89        6
409       6
17        6
1769      4
Length: 655, dtype: int64

In [98]:
# Define Graph
G = nx.Graph()

In [99]:
G.add_nodes_from(list_of_nodes_Road_toKeep)

In [100]:
len(set(list_of_edges))

56180

In [101]:
G.add_edges_from(list_of_edges)

In [102]:
# # Visualize Graph
# nx.draw_networkx(G)

In [103]:
# G.edges(nbunch=1682)

In [104]:
len(list(G.nodes))

655

In [ ]:
# nx.draw_networkx(G)

In [107]:
# # 459 is corr!
sig_1 = final_data_df['217']
# 619! 607!
list_of_node_crosscoef = []
for i in edge_series[217]:
    sig_2 = final_data_df[str(i)]
#     sig2 = sig2[sig2.betriebsdatum == '11.01.16']
#     sig2.sort_values(by='soll_an_von', inplace=True)
#     sig2 = sig2.groupby(by='intervals', as_index=False).mean()
#     sig2_y = np.array(sig2.actual_travel_time_n_target)
    list_of_node_crosscoef.append((i, np.corrcoef(sig_1, sig_2)[0][1]))
zlst = list(zip(*list_of_node_crosscoef))
list_of_node_crosscoef = pd.Series(zlst[1], index = zlst[0])
list_of_node_crosscoef = list_of_node_crosscoef.dropna().sort_values()
plt.scatter(list_of_node_crosscoef.index.values.tolist(), list_of_node_crosscoef.values.tolist())

# MAP animation

In [108]:
dict_gps = pd.Series(final.Mid_point_GPS_data.values,index=final.Route_Node_id).to_dict()
print(len(dict_gps))
for i, v in dict_gps.items():
    if i not in list_of_nodes_Road_toKeep:
        del dict_gps[i]
dict_gps
# final_data_df.map_data = 

655


{361: (47.3742375, 8.523571),
 122: (47.373825499999995, 8.52727),
 359: (47.375086499999995, 8.5189335),
 596: (47.376001, 8.515847),
 358: (47.374437, 8.522965500000002),
 1025: (47.3753815, 8.5248915),
 357: (47.373143999999996, 8.518861000000001),
 22: (47.3709435, 8.5151685),
 234: (47.370693, 8.521227499999998),
 18: (47.3701085, 8.5168865),
 479: (47.3706275, 8.522001),
 184: (47.371767000000006, 8.525169),
 232: (47.372018499999996, 8.525717),
 123: (47.372910000000005, 8.527973),
 804: (47.3693575, 8.533177),
 125: (47.3728805, 8.53256),
 173: (47.370988499999996, 8.536646000000001),
 124: (47.374964, 8.532430999999999),
 1331: (47.377219, 8.5322785),
 601: (47.375901, 8.535621500000001),
 602: (47.3760295, 8.5357765),
 590: (47.375406999999996, 8.525051000000001),
 1020: (47.377193500000004, 8.524452),
 1209: (47.377746, 8.526456),
 1485: (47.373036, 8.518905),
 43: (47.3744365, 8.5230255),
 593: (47.374936500000004, 8.5195005),
 589: (47.3744345, 8.522734),
 594: (47.37755, 

In [109]:
d = pd.DataFrame(dict_gps)
d.to_csv('gps_10min.csv')

In [110]:
final_data_df.to_csv('final_10min.csv')

In [111]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

# Finding correlation coefficient

In [115]:
sig1 = final[final.Route_Node_id == 210]
sig1 = sig1[sig1.betriebsdatum == '07.01.16']
sig1.sort_values(by='soll_an_von', inplace=True)
# sig1 = sig1
sig1 = sig1.groupby(by='intervals', as_index=False).mean()
# sig1

In [116]:
sig2 = final[final.Route_Node_id == 212]
sig2 = sig2[sig2.betriebsdatum == '07.01.16']
sig2.sort_values(by='soll_an_von', inplace=True)
sig2 = sig2.groupby(by='intervals', as_index=False).mean()
# sig2

In [117]:
sig1_X = np.array(sig1.intervals)
sig1_y = np.array(sig1.actual_travel_time_n_target)
sig1_y_zeromean = sig1_y - sig1_y.mean()

sig2_X = np.array(sig2.intervals)
sig2_y = np.array(sig2.actual_travel_time_n_target)
sig2_y_zeromean = sig2_y - sig2_y.mean()

plt.plot(sig1_X, sig1_y, sig2_X, sig2_y)

In [118]:
plt.plot(sig1_X, sig1_y_zeromean, sig2_X, sig2_y_zeromean)

In [119]:
from scipy.stats.stats import pearsonr
np.corrcoef(sig1_y_zeromean, sig2_y_zeromean)

array([[1.        , 0.10509196],
       [0.10509196, 1.        ]])

In [120]:
pearsonr_coef = pearsonr(sig1_y, sig2_y)
pearsonr_coef

(0.10509196192955886, 0.34140674866268106)

In [ ]:
# For the entire network - all signals connected to node 726:
# len(edge_series[726])

In [121]:
edge_series.value_counts()[:5]

381    178
217    178
213    177
386    177
218    177
dtype: int64

In [125]:
# 1283    218
# 1498    218
# 1497    212
# 611     212
# 622     212

%matplotlib
sig1 = np.array(final_data_df[str(213)])

list_of_node_crosscoef = []

for i in edge_series[213]:
    sig2 = np.array(final_data_df[str(i)])
    list_of_node_crosscoef.append((i, np.corrcoef(sig1, sig2)[0][1]))

zlst = list(zip(*list_of_node_crosscoef))
list_of_node_crosscoef = pd.Series(zlst[1], index = zlst[0])
list_of_node_crosscoef = list_of_node_crosscoef.dropna().sort_values()

plt.scatter(list_of_node_crosscoef.index.values.tolist(), list_of_node_crosscoef.values.tolist())

Using matplotlib backend: Qt5Agg


In [127]:
# Autocorrelation
def autocorr(x, t=1):
    return np.corrcoef(np.array([x[:-t], x[t:]]))
autocorr(sig2_y_zeromean)

array([[ 1.        , -0.15259675],
       [-0.15259675,  1.        ]])

In [128]:
%matplotlib
import seaborn as sns
corr = final_data_df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

Using matplotlib backend: Qt5Agg


In [131]:
ix = final_data_df.corr().sort_values('213', ascending=False).index
df_sorted = final_data_df.loc[:, ix]

In [132]:
import seaborn as sns
corr = df_sorted.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [133]:
plt.matshow(df_sorted.corr())
plt.show()

In [ ]:
# a = [int(i) for i in final_data_df.columns.values.tolist()]

# adj_mat = nx.to_numpy_matrix(G, nodelist=a)

# np.savetxt("PeMSD7_W_560_V1.csv", adj_mat, delimiter=",")

# Create the graph another way! V2

In [ ]:
# list_of_edges = []

# # iterate over each mid point aka road segment
# for i in list_of_nodes_Road:
#     # Look at every other road segment, if physical distance was less than 2.5 kilometers, add an edge
    
#     for key, value in Dict_List.items():
#         if Dict_List[int(i)] != value:
#             if vincenty(Dict_List[int(i)], value).meters < 2000:
#                 list_of_edges.append((int(i), key))

In [134]:
Nodes = list(set(new_final.Route_Node_id))


In [135]:
# G = nx.Graph()   # or DiGraph, MultiGraph, MultiDiGraph, etc
# G.add_weighted_edges_from([(0,1,3.0),(1,2,7.5)])
# nx.draw_networkx(G)

In [136]:
Dict_List_halt_ids = pd.Series(new_final.Route_Node_id_tuple.values,index=new_final.Route_Node_id).to_dict()
len(Dict_List)

655

In [137]:
# Create the new graph:
# Nodes = list(set(new_final.halt_id_x))
# print(len(Nodes))

# GPS is in Dict_List
Edges = []
for key, value in Dict_List_halt_ids.items():
    for key_2, value_2 in Dict_List_halt_ids.items():
        if key != key_2:
#             if value[0] == value_2[0] or value[0] == value_2[1] or value[1] == value_2[0] or value[1] == value_2[1]: 
            if value[0] == value_2[0] or value[1] == value_2[1]:    
                Edges.append((int(key), int(key_2), 1/(vincenty(Dict_List[int(key)], Dict_List[int(key_2)]).kilometers)))
    


C:\Users\arash\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  if sys.path[0] == '':


In [138]:
len(Edges)

206

In [139]:
New_G = nx.Graph()
New_G.add_nodes_from(list_of_nodes_Road_toKeep)
New_G.add_weighted_edges_from(Edges)

In [140]:
list(New_G.nodes)

[1407,
 1423,
 1070,
 1447,
 1066,
 619,
 1219,
 532,
 388,
 1406,
 517,
 608,
 616,
 1380,
 31,
 1498,
 1492,
 430,
 1589,
 495,
 1520,
 1236,
 596,
 1080,
 330,
 1272,
 1682,
 1280,
 1169,
 1860,
 631,
 64,
 544,
 210,
 1354,
 1109,
 1124,
 1428,
 292,
 350,
 1053,
 59,
 1349,
 834,
 1103,
 361,
 1025,
 1436,
 455,
 1832,
 1563,
 1309,
 882,
 429,
 1207,
 1226,
 60,
 69,
 1122,
 130,
 1646,
 1575,
 57,
 1119,
 1791,
 224,
 186,
 1662,
 919,
 910,
 1513,
 1573,
 598,
 1382,
 1786,
 1588,
 247,
 354,
 483,
 374,
 1451,
 883,
 264,
 1286,
 889,
 49,
 252,
 107,
 88,
 1547,
 638,
 1623,
 214,
 1426,
 1020,
 52,
 251,
 575,
 590,
 1535,
 58,
 1663,
 1223,
 594,
 90,
 165,
 639,
 359,
 1614,
 1400,
 169,
 1273,
 920,
 507,
 1469,
 804,
 212,
 1670,
 307,
 1393,
 1757,
 1651,
 428,
 1533,
 1799,
 1264,
 1557,
 991,
 1737,
 1471,
 1277,
 496,
 1402,
 1350,
 1348,
 1642,
 609,
 1383,
 1548,
 457,
 1805,
 21,
 1159,
 1266,
 279,
 1819,
 123,
 1506,
 125,
 54,
 452,
 163,
 48,
 1811,
 1609,
 34

In [141]:
nx.draw_networkx(New_G)

C:\Users\arash\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [ ]:
# # Get Line graph:
# New_G_line = nx.line_graph(New_G)

In [ ]:
# nx.draw(New_G_line)

In [ ]:
# print(len(list(New_G_line.nodes())))
# print(len(set(new_final.Route_Node_id)))

# print(len(list(New_G_line.edges())))
# # print(len(set(new_final.Route_Node_id)))

In [ ]:
set1 = set(list(New_G.node))
set2 = set(new_final.Route_Node_id)
set2 - set1

# Remove nodes with no connections


In [ ]:
len(New_G.nodes)

In [ ]:
Connected_New_G = max(nx.connected_component_subgraphs(New_G), key=len)
nx.draw_networkx(Connected_New_G)

In [ ]:
index = [str(timedelta(seconds=i * time_period + starting_time)) for i in set(final.intervals)] * 3

In [142]:
from datetime import timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

import folium
import folium.plugins as plugins
import numpy as np

df = pd.read_csv('final_10min.csv', nrows=252)

df.interpolate(inplace=True, axis=1)

print(df.isna().sum().sum())

df_2 = pd.read_csv('gps_10min.csv')

df = (df - df.mean()) / (df.std() ** 1.2)

df_2.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)

# Outer list:
list_df_gps = df_2.columns.values.tolist()
OuterList = []
for index, row in df.iterrows():
    Mid_List = []
    for j in list_df_gps:
        Time_List = df_2[j].values.tolist()
#         print(type(j))
#         print(row[0])
        Time_List.append(row[j])
        Mid_List.append(Time_List)
    OuterList.append(Mid_List)

m = folium.Map([47.36179377478453, 8.572671632662107], tiles='stamentoner', zoom_start=12)

list_of_id = df_2.values
list_of_id = list(zip(list_of_id[0], list_of_id[1]))

for j in list_of_id:
    folium.Circle(j, popup='<strong>Location One</strong>', radius=20).add_to(m)

index = [str(timedelta(seconds=i * time_period + starting_time)) for i in set(final.intervals)] * int(df.shape[0]/len(set(final.intervals)))

hm = plugins.HeatMapWithTime(OuterList, index=index)

hm.add_to(m)

m.save('index.html')

0


In [143]:
list_of_id = df_2.values
list_of_id = list(zip(list_of_id[0], list_of_id[1]))

In [144]:
# Visualize the graph with the heatmap

In [145]:
m3 = folium.Map(location=[47.417118, 8.568561], zoom_start=12, tiles="Stamen Terrain")
fahrweg_id_to_look_for = final[final.halt_id_x==1565].fahrweg_id.value_counts().index
for i in fahrweg_id_to_look_for:

    a = final[final.fahrweg_id == i]
    if 1565 in set(a.halt_id_x):
        a = a.groupby(by='seq_von').agg(np.mean)

        if a.empty or len(a) < 2:
            continue

        list_of_corrd = list(zip(a.GPS_Latitude_x_x, a.GPS_Longitude_x_x))

        list_of_corrd_halt = list(zip(a.GPS_LAN_STELLE_FROM, a.GPS_LON_STELLE_FROM))

        color = [i for i in range(len(list_of_corrd) - 1)]
        if not color:
            continue
        folium.ColorLine(list_of_corrd, colors=color, colormap=('#00008B', '#00BFFF'), weight=4).add_to(m3)
        for j in list_of_corrd_halt:
            folium.Circle(j, popup='<strong>Location One</strong>', tooltip=tooltip, radius=100).add_to(m3)


        for j in list_of_corrd:
            folium.Marker(j, popup='<strong>Location One</strong>', tooltip=tooltip, icon=folium.Icon(icon='cloud', color=list_of_colors[i % len(list_of_colors)])).add_to(m3)
    index_of_color += 1

# list_of_punkts_in_stelle = list(zip(list(set(final[final.halt_id_x == 1565].GPS_Latitude_x_x)), list(set(final[final.halt_id_x == 1565].GPS_Longitude_x_x))))
# for i in list_of_punkts_in_stelle:
#     folium.Marker([i[0], i[1]], popup='<strong>Location One</strong>', tooltip=tooltip).add_to(m2)


# print(list_of_corrd)
# Create Markers
#     folium.Marker(list_of_corrd, popup='<strong>Location One</strong>', tooltip = tooltip, icon=folium.Icon(icon='cloud', color='purple')).add_to(m2)
folium.Marker([47.367029, 8.545065], popup='<strong>Location One</strong>', tooltip=tooltip).add_to(m3)

# print(list_of_corrd)


# Make map
m3.save('map_Graph_Connections.html')

NameError: name 'tooltip' is not defined

# Save Files

In [147]:
a = [int(i) for i in final_data_df.columns.values.tolist()]

adj_mat = nx.to_numpy_matrix(G, nodelist=a)

In [148]:
final_data_df.to_csv('PeMSD7_V_655.csv', header=False, index=False)

In [149]:
np.savetxt("PeMSD7_W_655.csv", adj_mat, delimiter=",")

In [ ]:
New_G.edges(data=True)